In [1]:
import json
import os
import pickle
from pathlib import Path
from joblib import Parallel, delayed

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import rioxarray
import seaborn as sns
from tqdm import tqdm
from xrspatial.multispectral import ndvi, savi
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay)
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss

The files are stored on a spinning hard disk, so I need to move them to the data dir.

In [ ]:
%%bash
